In [ ]:
using Plots,LaTeXStrings
default(markersize=3,linewidth=1.5)
using LightGraphs,GraphPlot
using Images,TestImages
using DataFrames,JLD
using LinearAlgebra
#include("FNC.jl");

# Example 7.1.4

Here is the adjacency matrix of a "small-world" network on 200 nodes. Each node is connected to 4 neighbors, and then some edges are randomly changed to distant connections. 

In [ ]:
g = watts_strogatz(200,4,0.06)
gplot(g)

In [ ]:
g

In [ ]:
g.fadjlist

In [ ]:
g.ne

In [ ]:
vertices(g)

In [ ]:
edges(g)

In [ ]:
collect(edges(g))

The adjacency matrix for this graph reveals the connections as mostly local (i.e., the nonzeros are near the diagonal).

In [ ]:
A = adjacency_matrix(g,Float64)

In [ ]:
Matrix(A)

In [ ]:
spy(A,m=1.5,color=:black,title="Adjacency matrix",leg=:none,size=(500,500))
xlims!(-10,210); ylims!(-10,210)

In [ ]:
sum(A,dims=2)

# Example 7.1.5

We will use the `Images` package for working with images. We also load here the `TestImages` package for a large library of well-known standard images.

In [ ]:
img = testimage("peppers")

The details vary by image type, but for the most part an image is an array of color values.

In [ ]:
size(img),eltype(img)

In [ ]:
2^8

The elements here have four values, for red, green, blue, and alpha (opacity). We can convert each of those "planes" into an ordinary matrix.

In [ ]:
R = red.(img)

In [ ]:
R[1:5,1:5]

The values above go from zero (no red) to one (full red). It may also be convenient to convert the image to grayscale, which has just one "layer" from zero (black) to one (white). 

In [ ]:
G = Gray.(img)

In [ ]:
A = @. gray(Gray(img))

In [ ]:
A[1:5,1:5]

Finally, we can save an image locally for reloading later.

In [ ]:
save("peppers.png",Gray.(img))

In [ ]:
load("peppers.png")

# Example 7.2.1

The `eigvals` command will return just the eigenvalues, as a vector. 

In [ ]:
A = pi*ones(2,2)

In [ ]:
lambda = eigvals(A)

If we also want the eigenvectors (returned as the matrix $V$), we use `eigen`.

In [ ]:
lambda,V = eigen(A)

In [ ]:
lambda

In [ ]:
V

We can check the fact that this is an EVD.

In [ ]:
D = diagm(lambda)

In [ ]:
D = Diagonal(lambda)

In [ ]:
opnorm( A - V*D/V )      # "/V" is like "*inv(V)""

Even if the matrix is not diagonalizable, `eigen` will run successfully, but the matrix ${V}$ will not be invertible.

In [ ]:
lambda,V = eigen([1 1;0 1])

In [ ]:
rank(V)

# Example 7.2.2

We will confirm the Bauer-Fike theorem on a triangular matrix. These tend to be far from normal. 

In [ ]:
n = 15
lambda = 1:n
A = triu( ones(n)*lambda' )

The Bauer-Fike theorem provides an upper bound on the condition number of these eigenvalues.

In [ ]:
lambda,V = eigen(A)

In [ ]:
cond(V)

The theorem suggests that eigenvalue changes may be up to 7 orders of magnitude larger than a perturbation to the matrix. A few random experiments show that effects of nearly that size are not hard to observe.

In [ ]:
for k = 1:3
    E = randn(n,n);  E = 1e-7*E/opnorm(E)
    mu = eigvals(A+E)
    @show max_change = norm( sort(mu)-lambda, Inf )
end

In [ ]:
E = randn(n,n);  E = 1e-7*E/opnorm(E)
mu = eigvals(A+E)

In [ ]:
R = A*V - V*Diagonal(lambda)

In [ ]:
[cond(V)*norm(R[:,i])/norm(V[:,i]) for i=1:n]

In [ ]:
lambda - collect(1:n)

# Example 7.2.3

Let's start with a known set of eigenvalues and an orthogonal eigenvector basis.

In [ ]:
D = diagm([-6,-1,2,4,5])

In [ ]:
V,_ = qr(randn(5,5))  # compute a random orthogonal matrix V
A = Symmetric(V*D*V')    # note that V' = inv(V)

In [ ]:
issymmetric(A)

In [ ]:
eigvals(A)

Now we will take the QR factorization and just reverse the factors.

In [ ]:
Q,R = qr(A)
A = Symmetric(R*Q)

It turns out that this is a similarity transformation, so the eigenvalues are unchanged.

In [ ]:
eigvals(A)

What's remarkable is that if we repeat the transformation many times, the process converges to $D$. 

In [ ]:
for k = 1:40
    Q,R = qr(A)
    A = Symmetric(R*Q)
end
A

# Example 7.3.2

We verify some of the fundamental SVD properties using the built-in Julia command `svd`.

In [ ]:
A = [i^j for i=1:5, j=0:3]

In [ ]:
U,sigma,V = svd(A)

In [ ]:
V

In [ ]:
?diagm

In [ ]:
S = diagm(sigma)

In [ ]:
U*S*V'

In [ ]:
?svd

In [ ]:
U, sigma, V = svd(A, full=true)

In [ ]:
S = diagm(5, 4, sigma)

In [ ]:
U*S*V'

Note that while the "full" SVD has a square $U$, the "thin" form is the default. Here the columns are orthonormal even though $U$ is not square.

In [ ]:
@show size(U),opnorm(U'*U - I);

In [ ]:
@show size(V),opnorm(V'*V - I);

In [ ]:
sigma

In [ ]:
@show opnorm(A),sigma[1];

In [ ]:
@show cond(A), sigma[1]/sigma[end];

# Example 7.4.1

The following matrix is not hermitian.

In [ ]:
A = [0 2; -2 0]

It has an eigenvalue decomposition with a unitary matrix of eigenvectors, though, so it is normal. 

In [ ]:
lambda,V = eigen(A)

In [ ]:
opnorm( V'*V - I )

The eigenvalues are pure imaginary.

In [ ]:
lambda

The singular values are the complex magnitudes of the eigenvalues.

In [ ]:
svdvals(A)

# Example 7.4.2

We construct a real symmetric matrix with known eigenvalues by using the QR factorization to produce a random orthogonal set of eigenvectors. 

In [ ]:
n = 30;
lambda = 1:n 

D = diagm(lambda)
V,R = qr(randn(n,n))   # get a random orthogonal V
A = Symmetric(V*D*V')

The condition number of these eigenvalues is one. Thus the effect on them is bounded by the norm of the perturbation to $A$. 

In [ ]:
E = Symmetric(randn(n,n)); E = 1e-4*E/opnorm(E)
mu = sort(eigvals(A+E))
@show max_change = norm(mu-lambda,Inf)

In [ ]:
A+E

In [ ]:
mu

In [ ]:
for k = 1:3
    E = Symmetric(randn(n,n)); E = 1e-4*E/opnorm(E)
    mu = sort(eigvals(A+E))
    @show max_change = norm(mu-lambda,Inf)
end

# Example 7.4.3

We construct a symmetric matrix with a known EVD.

In [ ]:
n = 20;
lambda = 1:n 

D = diagm(lambda)
V,R = qr(randn(n,n))   # get a random orthogonal V
A = Symmetric(V*D*V')

The Rayleigh quotient of an eigenvector is its eigenvalue.

In [ ]:
Rayleigh(x) = (x'*A*x)/(x'*x)
Rayleigh(V[:,7])

The Rayleigh quotient's value is much closer to an eigenvalue than its input is to an eigenvector. In this experiment, each additional digit of accuracy in the eigenvector estimate gives two more digits to the eigenvalue estimate.

In [ ]:
delta = @. 1 ./10^(1:4)

In [ ]:
delta = [10.0^-i for i=1:4]

In [ ]:
quotient = zeros(size(delta))

In [ ]:
enumerate(delta)

In [ ]:
for (k,del) = enumerate(delta)
    e = randn(n);  e = e/norm(e)  # random normal vector
    x = V[:,7] + del*e
    quotient[k] = Rayleigh(x)
end
DataFrame(perturbation=delta,RQminus7=quotient.-7)

# Example 7.5.1

We make an image from some text, then reload it as a matrix.

In [ ]:
plot([],[],leg=:none,annotations=(0.5,0.5,text("Hello world",44,:center,:middle)),
    grid=:none,frame=:none)

savefig("hello.png")
img = load("hello.png")

In [ ]:
typeof(img)

In [ ]:
typeof(Gray.(img))

In [ ]:
Gray.(img)

In [ ]:
A = Float64.(Gray.(img))

In [ ]:
m,n = size(A)

In [ ]:
A[180:185,265:270]

In [ ]:
Gray.(A[180:185,265:270])

In [ ]:
Gray.(A[175:220,260:295])

Next we show that the singular values decrease exponentially, until they reach zero (more precisely, are about $\sigma_1 \varepsilon_\text{mach}$). For all numerical purposes, this determines the rank of the matrix.

In [ ]:
U,sigma,V = svd(A);
sigma

In [ ]:
scatter(sigma,
    title="Singular values",xaxis=(L"i"), yaxis=(:log10,L"\sigma_i"),leg=:none )

In [ ]:
r = findlast(sigma/sigma[1] .> 10*eps())  # Numerical rank

In [ ]:
r = findlast(sigma .> 1e-3)

In [ ]:
sigma[r-1:r+1]

In [ ]:
Ar = U[:,1:r]*diagm(sigma[1:r])*V[:,1:r]'
Gray.(Ar)

In [ ]:
extrema(abs.(A - Ar))

In [ ]:
abserr = maximum(abs.(A - Ar))

In [ ]:
extrema(abs.(A - Ar)./abserr)

In [ ]:
Gray.(abs.(A - Ar)./abserr)

In [ ]:
compression = r*(m+n+1) / (m*n)

The rapid decrease suggests that we can get fairly good low-rank approximations. 

In [ ]:
k = 8
Ak = U[:,1:k]*diagm(sigma[1:k])*V[:,1:k]'
Gray.(Ak)

In [ ]:
sigma[1:8]

In [ ]:
k = 1
Gray.(1*sigma[k]*U[:,k]*V[:,k]')

Consider how little data is needed to reconstruct these images. For rank 8, for instance, we have 8 left and right singular vectors plus 8 singular values, for a compression ratio of better than 25:1.  

In [ ]:
compression = 8*(m+n+1) / (m*n)

In [ ]:
A8 = U[:,1:8]*diagm(sigma[1:8])*V[:,1:8]'
Gray.(A8)

# Example 7.5.2

This matrix describes the votes on bills in the 111th session of the United States Senate. (The data set was obtained from voteview.com.) Each row is one senator and each column is a vote item.

In [ ]:
vars = load("voting.jld")

In [ ]:
Rep = vec(vars["Rep"])
Dem = vec(vars["Dem"])
Ind = vec(vars["Ind"])
length.([Rep, Dem, Ind])

In [ ]:
A = vars["A"]

In [ ]:
unique(A)

In [ ]:
m,n = size(A)

If we visualize the votes (white is "yea" (1), black is "nay" (-1), and gray is anything else (0)), we can see great similarity between many rows, reflecting party unity.

In [ ]:
heatmap(A,color=:greys,
    title="Votes in 111th U.S. Senate",xlabel="bill",ylabel="senator")

In [ ]:
heatmap(A[Rep,:], color=:grays, layout=(2,1),
    title="Votes in 111th U.S. Senate", ylabel="Republican senator")
heatmap!(A[Dem,:], color=:greys, subplot=2,
    xlabel="bill", ylabel="Democratic senator")

We use singular value "energy" to quantify the decay rate of the values. 

In [ ]:
U,sigma,V = svd(A)
sigma

In [ ]:
tau = cumsum(sigma.^2) / sum(sigma.^2)

In [ ]:
scatter(tau[1:16],label="",
    xaxis=("k"), yaxis=(L"\tau_k"), title="Fraction of singular value energy")

The first and second singular triples contain about 58% and 17% respectively of the energy of the matrix. All others have far less effect, suggesting that the information is primarily two-dimensional. The first left and right singular vectors also contain interesting structure.

In [ ]:
U[:,1]

In [ ]:
scatter(U[Dem,1], Dem, color=:blue, label="D", legend=:outertopright,
    ylabel="senator", title="left singular vector", size=(400,600))
scatter!(U[Rep,1], Rep, color=:red, label="R")
scatter!(U[Ind,1], Ind, color=:yellow, label="I")

In [ ]:
V[:,1]

In [ ]:
scatter(1:n, V[:,1], label="",
    xlabel="bill", title="right singular vector", size=(700,300))

Both vectors have values greatly clustered near $\pm C$  for a constant $C$. These can be roughly interpreted as how partisan a particular senator or bill was, and for which political party.   Projecting the senators' vectors into the first two $\mathbf{V}$-coordinates gives a particularly nice way to reduce them to two dimensions. Political scientists label these dimensions "partisanship" and "bipartisanship." Here we color them by actual party affiliation (also given in the data file): red for Republican, blue for Democrat, and yellow for independent. 

In [ ]:
x1 = A*V[:,1]  # = sigma[1]*U[:,1]

In [ ]:
x2 = A*V[:,2] # = sigma[2]*U[:,2]

In [ ]:
scatter(x1[Dem], Dem, color=:blue, label="D", legend=:outertopright,
    layout=(1,2), subplot=1, size=(700,600),
    title=L"\sigma_1 u_1", ylabel="senator" ,xlabel="partisanship")
scatter!(x1[Rep], Rep, color=:red, label="R")
scatter!(x1[Ind,], Ind, color=:yellow, label="I")

scatter!(x2[Dem], Dem, color=:blue, label="D", legend=:outertopright, 
    subplot=2,
    title=L"\sigma_2 u_2", xlabel="bipartisanship")
scatter!(x2[Rep], Rep, color=:red, label="R", subplot=2)
scatter!(x2[Ind], Ind, color=:yellow, label="I", subplot=2)

In [ ]:
scatter(x1[Dem],x2[Dem],color=:blue,label="D",
    xaxis=("partisanship"),yaxis=("bipartisanship"),title="111th US Senate in 2D" )
scatter!(x1[Rep],x2[Rep],color=:red,label="R")
scatter!(x1[Ind],x2[Ind],color=:yellow,label="I")